In [22]:
import joblib
import os
import sys

sys.path.append(os.path.abspath(os.path.join('..')))
from steps.ingest_data import ingest_data
from steps.clean_data import clean_data
from steps.model_train import train_model, save_model
from steps.evaluation import evaluate_model


In [26]:
from sklearn.calibration import calibration_curve

In [20]:
path_nb = "C:/Users/rober/OneDrive/Escriptori/DataSets/xgboost-calibration/XGBoost-and-Model-Calibration/"
xgb_model = joblib.load(f"{path_nb}saved_models/xgb_model.joblib")

In [23]:
# get params from model
churn_data = ingest_data(data_path=f"{path_nb}data/Churn_Modelling.csv")
X_train, X_test, y_train, y_test = clean_data(churn_data)
params = xgb_model.get_params()

In [24]:
predictions = xgb_model.predict_proba(X_test)
prob_pos = predictions[:, 1]

In [27]:
fraccion_positivos, media_prob_predicha = calibration_curve(y_test, prob_pos, n_bins=20)